# Context
This notebook drives the training process for different models.

In [9]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../project.env")
sys.path.append(os.environ["PYTHONPATH"])

In [10]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAIN.architecture.BERT.bert as bert

# Models

## Setup

In [11]:
import torch
import gc

In [12]:
TRAIN_classes = 'ten-classes'
exploringBERT = bert.BERT_TINY
exploring_num_components = [9]
exploring_base_num_epochs = 80
exploring_batch_size = 1024

## Train

In [13]:
sup.bert_score_tracker = []

In [14]:
for data_unit in [sup.DATA_S_PF, sup.DATA_S_PV]:
  bert.find_best(data_unit=data_unit, 
                  label_col=sup.class_numeric_column,
                  class_list=TRAIN_classes, 
                  batch_size=exploring_batch_size,
                  N_CANDIDATES=exploring_num_components,
                  LOADABLE_CANDIDATES=[exploringBERT],
                  base_num_epochs=exploring_base_num_epochs
                  )
  
  gc.collect()
  if torch.backends.mps.is_available():
    torch.mps.empty_cache()

gc.collect()
if torch.backends.mps.is_available():
  torch.mps.empty_cache()

{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 9}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 7200}


0.8648434403487911
not best... at 2025-06-15_23:52:14



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 75}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 7200}


0.9603646452635751
not best... at 2025-06-16_00:29:53



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 9}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 7200}


0.9437177962742767
not best... at 2025-06-16_01:07:20



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 7200}


0.9730479587792311
not best... at 2025-06-16_01:40:27



Data Unit: Spf
Best score: 0.994140625
Best data config: {'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'two-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
Best train config: {'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 7200}
{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 9}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e

0.6824644549763034
not best... at 2025-06-16_02:16:35



{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 75}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 28800}


0.8388625592417062
not best... at 2025-06-16_02:39:45



{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 9}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 28800}


0.8199052132701422
not best... at 2025-06-16_03:07:26



{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 28800}


0.957345971563981
not best... at 2025-06-16_03:39:58



Data Unit: Spv
Best score: 0.9767441860465116
Best data config: {'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
Best train config: {'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-tiny-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 28800}


# Keep metrics

In [15]:
import pandas as pd
from datetime import datetime

In [16]:
best_scores_df = pd.DataFrame(sup.bert_score_tracker, columns=sup.bert_scores_columns)
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, 
                                          sup.TRAIN_BERT_CODE, exploringBERT))
best_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes,
                                       sup.TRAIN_BERT_CODE, exploringBERT,
                                       f"BERTbest-{now}.csv"), index=False)